In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from datetime import datetime

In [8]:
with open('data/R1_10-23_IVS_MOREDELTA_ANN.pkl', 'rb') as file:
    ivs = pickle.load(file)

In [9]:
def ivs_model(params, m, t):
    a0, a1, a2, a3, a4, a5 = params
    return (a0 + (a1 * m) + (a2 * t) + (a3 * m**2) + (a4 * t**2) + (a5 * m * t))

def objective_function(params, m, t, observed_iv):
    model_ivs = ivs_model(params, m, t)
    return np.sqrt(np.mean((observed_iv - model_ivs) ** 2))

In [10]:
def dfw_interpolation(ivs):
    columns_to_drop = [col for col in ivs.columns if (float(col) < 0.75) or (float(col) > 1.25)]
    ivs = ivs.drop(columns=columns_to_drop)
    
    o_moneyness, o_dte = np.meshgrid(ivs.columns, ivs.index)
    o_moneyness = o_moneyness.ravel()
    o_dte = o_dte.ravel()
    o_ivs = ivs.values.ravel()
    unravelled_ivs = pd.DataFrame()
    unravelled_ivs['m'] = o_moneyness
    unravelled_ivs['t'] = o_dte
    unravelled_ivs['iv'] = o_ivs
    unravelled_ivs = unravelled_ivs.dropna(subset=['iv'])

    initial_params = np.zeros(6)
    result = minimize(objective_function, initial_params, args=(unravelled_ivs['m'], unravelled_ivs['t'], unravelled_ivs['iv']))
    return result.x, result.fun

In [11]:
standard_ivs = {}
moneyness_list = [0.8, 0.84, 0.88, 0.92, 0.96, 1.0, 1.04, 1.08, 1.12, 1.16, 1.20]
dte_list = [15.0/365.0, 30.0/365.0, 60.0/365.0, 91.0/365.0, 122.0/365.0, 152.0/365.0, 182.0/365.0, 273.0/365.0, 365.0/365.0, 547.0/365.0, 730.0/365.0]
rmse_sum = 0.0
for date in ivs:
    params, rmse = dfw_interpolation(ivs[date])
    rmse_sum += rmse
    a0, a1, a2, a3, a4, a5 = params
    standard_ivs[date] = [pd.DataFrame(index=dte_list, columns=moneyness_list), params]
    for dte in standard_ivs[date][0].index:
        for moneyness in standard_ivs[date][0].columns:
            iv = a0 + (a1 * moneyness) + (a2 * dte) + (a3 * moneyness**2) + (a4 * dte**2) + (a5 * moneyness * dte)
            #print(f'Moneyness: {moneyness} | DTE: {dte} | IV: {iv}')
            standard_ivs[date][0].loc[dte, moneyness] = max(0.01, iv)
    standard_ivs[date][0] = standard_ivs[date][0].apply(pd.to_numeric, errors='ignore')
    print(date)
print(rmse_sum/len(ivs))

2010-02-08
2010-02-09
2010-02-10
2010-02-11
2010-02-12
2010-02-16
2010-02-18
2010-02-19
2010-02-23
2010-02-26
2010-03-02
2010-03-09
2010-02-17
2010-03-10
2010-03-04
2010-03-08
2010-02-24
2010-02-22
2010-02-25
2010-03-05
2010-03-01
2010-03-03
2010-03-12
2010-03-11
2010-03-16
2010-03-17
2010-03-22
2010-03-23
2010-03-24
2010-03-25
2010-03-26
2010-03-15
2010-03-18
2010-03-19
2010-03-30
2010-03-31
2010-04-01
2010-04-08
2010-04-09
2010-03-29
2010-04-05
2010-04-07
2010-04-06
2010-04-13
2010-04-16
2010-04-19
2010-04-21
2010-04-22
2010-04-27
2010-04-29
2010-04-30
2010-05-04
2010-05-06
2010-05-07
2010-05-10
2010-05-14
2010-05-17
2010-05-18
2010-05-20
2010-05-24
2010-05-26
2010-05-27
2010-06-07
2010-06-08
2010-06-09
2010-06-11
2010-05-13
2010-05-21
2010-04-26
2010-05-11
2010-05-25
2010-06-04
2010-06-03
2010-04-23
2010-05-12
2010-06-01
2010-06-10
2010-05-03
2010-04-15
2010-04-14
2010-05-28
2010-04-12
2010-05-05
2010-06-02
2010-04-20
2010-04-28
2010-06-15
2010-06-16
2010-06-18
2010-06-22
2010-06-23

In [19]:
std_ivs = {date: standard_ivs[date][0] for date in sorted(standard_ivs, key=lambda date: datetime.strptime(date, '%Y-%m-%d'))}

In [20]:
std_ivs['2019-09-06']

,0.80,0.84,0.88,0.92,0.96,1.00,1.04,1.08,1.12,1.16,1.20
0.041096,0.448382,0.376319,0.311553,0.254083,0.203911,0.161035,0.125456,0.097174,0.076189,0.062500,0.056108
0.082192,0.443462,0.372130,0.308094,0.251355,0.201914,0.159768,0.124920,0.097368,0.077114,0.064156,0.058494
0.164384,0.433758,0.363886,0.301312,0.246035,0.198054,0.157370,0.123983,0.097892,0.079099,0.067602,0.063401
0.249315,0.423919,0.355558,0.294493,0.240726,0.194255,0.155081,0.123203,0.098623,0.081339,0.071352,0.068662
0.334247,0.414273,0.347422,0.287867,0.235610,0.190648,0.152984,0.122617,0.099546,0.083772,0.075295,0.074114
0.416438,0.405122,0.339732,0.281638,0.230842,0.187342,0.151138,0.122232,0.100622,0.086310,0.079294,0.079574
0.498630,0.396151,0.332222,0.275589,0.226254,0.184215,0.149473,0.122028,0.101879,0.089028,0.083473,0.085215
0.747945,0.370041,0.310544,0.258344,0.213440,0.175834,0.145524,0.122511,0.106795,0.098375,0.097252,0.103426
1.000000,0.345330,0.290314,0.242595,0.202172,0.169047,0.143218,0.124685,0.113450,0.109511,0.112869,0.123524
1.498630,0.301441,0.255289,0.216434,0.184876,0.160614,0.143649,0.133981,0.131610,0.136536,0.148758,0.168277


In [21]:
with open('data/R2_STD_IVS_DFW_SORTED.pkl', 'wb') as file:
    pickle.dump(std_ivs, file)